In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta, time, datetime


import warnings
from IPython.core.display import display, HTML
warnings.filterwarnings("ignore") # suppress warnings because theyre annoying and i like living on the edge
pd.set_option('display.max_columns',100) # so i can see all columns
np.set_printoptions(linewidth = 220) # increase numpy array print
display(HTML("<style>.container { width:80% !important; }</style>")) # widen jupyter notebook blocks

In [ ]:
conn_cred = {
    "dbservername":"localhost",
    "dbname":"main_schema",
    "dbuser":"minx",
    "dbpassword":"!xobILE!!!111!"
}

In [ ]:
import pymysql as MySQLdb

In [ ]:

def mysql_conn(db_conn_info):
    '''
    Call to connect to the database
    '''
    try:
        dbconn_ = MySQLdb.connect(host=db_conn_info['dbservername'],
                    db = db_conn_info['dbname'],
                    user = db_conn_info['dbuser'],
                    passwd = db_conn_info['dbpassword']
                    )
        cursor_ = dbconn_.cursor()
        return dbconn_, cursor_

    except Exception as e:
        print('Error: ' + str(e))

def db_conn_close(dbconn, cursor):
    # closing connections to free up sockets
    dbconn.commit()
    cursor.close()
    dbconn.close()
    
    
def run_query(creds, query):
    conn, cursor = mysql_conn(creds)

    try:
        cursor.execute(query)
        rows = cursor.fetchall()
    except Exception as e:
        print('Error in running query: ' + str(e))
    finally:
        db_conn_close(conn, cursor)
    return rows

In [ ]:
query_ = 'SELECT * FROM {dbname}.bar_15min'.format(dbname = conn_cred['dbname'])
rows = run_query(conn_cred,query_)

In [ ]:
df = pd.DataFrame(rows, columns = ['symbol','datetime','open','close','high','low','volume'])

In [ ]:
# df['delta'] = df['close'] - df['open']
vals = df['close'].astype(float).values[1:] - df['close'].astype(float).values[:-1]
vals = np.append(np.array([0]),vals)
df['delta'] = vals

In [ ]:
df['midpoint'] = (df['high'] + df['low']) / 2
# df['midpoint'] = (df['open'] + df['close']) / 2

vals = df['midpoint'].astype(float).values[1:] - df['midpoint'].astype(float).values[:-1]
vals = np.append(np.array([0]),vals); vals
df['midpoint_delta'] = vals

In [ ]:
# def calculate_rsi(vals, prevU = 0, prevD = 0, n = 9):
# #     alpha = 2 / (n+1) # exponential method
#     alpha = 1
#     U = np.sum(vals * (vals > 0).astype(int)) / n
#     D = -1 * np.sum(vals * (vals < 0).astype(int)) / n
#     avgU = alpha * U + (1 - alpha) * prevU
#     avgD = alpha * D + (1 - alpha) * prevD

#     prevU = avgU
#     prevD = avgD
#     rs = avgU / avgD
#     rsi = 100.0 - 100.0 / (1 + rs)
#     return rsi, prevU, prevD

def calculate_rsi(val, prevU = 0, prevD = 0, n = 9):
    if val > 0:
        avgU = (prevU*(n-1) + val) / n
        avgD = prevD*((n-1)/n)
    else:
        avgU = prevU*((n-1)/n)
        avgD = (prevD*(n-1) - val) / n
        
    rs = avgU / avgD
    rsi = 100.0 - 100.0 / (1 + rs)
    return rsi, avgU, avgD

In [ ]:
rsi_size = 9
n_iter = df.shape[0] - rsi_size + 1
rsi_list = []

# first rsi

vals = df['delta'].astype(float).values[:rsi_size]
prevU = np.sum(vals * (vals > 0).astype(int)) / rsi_size
prevD = -1 * np.sum(vals * (vals < 0).astype(int)) / rsi_size
rs = prevU / prevD
rsi_ = 100.0 - 100.0 / (1 + rs)
rsi_list.append(rsi_)

for i in range(rsi_size,df.shape[0]):
    val = df['delta'].iloc[i]
    rsi_, prevU, prevD = calculate_rsi(val, prevU, prevD)
    rsi_list.append(rsi_)
    

In [ ]:
rsi_list

In [ ]:
plt.figure(figsize = [15,5])
plt.plot(range(n_iter), rsi_list)
plt.ylim(0,100)
plt.show()

In [ ]:
temp = df[['datetime','close','delta']].copy()
temp['rsi'] = [0] * (rsi_size-1) + rsi_list
temp